In [5]:
from bs4 import BeautifulSoup
import requests
import selenium
import random
from time import sleep
import pandas as pd

In [6]:
# logging is important
logger = logging.getLogger()
dstamp = datetime.now().strftime('%m_%d')
fhandler = logging.FileHandler(filename= dstamp + 'session.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)


# Constants

In [9]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0'} 

In [10]:
baseurl =  'https://data.gov.ph/sites/default/files/'

In [14]:
def urlToBs(url, headers):
    sleep(random.uniform(0,0.5))
    pg = requests.get(url, headers)
    sleep(1)
    bs = BeautifulSoup(pg.content, 'html.parser')
    return bs

In [31]:
def sp_to_lturl(bsPage):
    return [a['href'] for a in bsPage.find_all("a", {'class':"btn btn-primary data-link"})] 

In [32]:
purl =  'https://data.gov.ph/?q=dataset/climatological-normal-values'

In [21]:
bsPage = urlToBs(purl, headers)

In [33]:
hturllt = sp_to_lturl(bsPage) 

In [45]:
def fileOut(content, fname):
    fout = open(fname, 'wb')
    fout.write(content)
    fout.close()

In [47]:
r = requests.get(hturllt[0], allow_redirects=True)

In [52]:
hturllt[0].split('/')[-1]

'pagasanormvalalabatquez1999-2010.csv'

In [55]:
for url in hturllt:
    r = requests.get(url, allow_redirects=True)
    fileOut(r.content, './precipCSV/' + url.split('/')[-1])

In [79]:
def readAllCsv(lt, fp):
    dtdfImported = {}
    for fname in lt:
        logging.info(fname)
        dtdfImported[fname] = pd.read_csv(fp + fname)
        locstr = fname.split('.')[0][13:] 
        dtdfImported[fname]['location'] = locstr
    return dtdfImported

In [65]:
ltfname = [url.split('/')[-1] for url in hturllt]

In [86]:
dtdfImported =  readAllCsv(ltfname, './precipCSV/')

In [87]:
len('pagasanormval')

13

In [89]:
ltdfImported = [df for df in  dtdfImported.values()]

In [90]:
dfFinal = pd.concat(ltdfImported)

In [93]:
dfFinal.shape

(715, 23)

In [98]:
dfFinal

,month,rainfall_amount,rainfall_number_of_rd,temp_max,temp_min,temp_mean,temp_dry_bulb,temp_wet_bulb,temp_dew_pt,vapor_pressure_mbs,...,wind_spd_mps,cloud_amount_okta,number_of_days_with_tstm,number_of_days_with_ltng,location,rel_hum,mslp,vapor_pressure,,wind_spd
0,January,250.2,21,28.3,21.9,25.1,25.5,23.6,22.9,27.8,...,4.0,7,0,0,alabatquez1999-2010,NaN,NaN,NaN,NaN,NaN
1,February,151.8,14,28.9,21.9,25.4,25.8,23.8,23.0,28.1,...,3.0,6,0,0,alabatquez1999-2010,NaN,NaN,NaN,NaN,NaN
2,March,127.4,11,30.2,22.6,26.4,26.7,24.5,23.7,29.2,...,3.0,5,2,1,alabatquez1999-2010,NaN,NaN,NaN,NaN,NaN
3,April,91.9,9,31.8,23.4,27.6,28.0,25.4,24.5,30.6,...,2.0,5,5,3,alabatquez1999-2010,NaN,NaN,NaN,NaN,NaN
4,May,114.1,11,32.8,23.8,28.3,28.7,26.0,25.1,31.7,...,2.0,6,14,12,alabatquez1999-2010,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,September,482.8,21,26.2,20.4,23.3,23.0,22.1,21.8,26.1,...,3.0,7,14,13,anayrizal1999-2012,NaN,NaN,NaN,NaN,NaN
9,October,276.5,18,26.2,20.4,23.3,22.9,21.8,21.4,25.4,...,3.0,6,10,13,anayrizal1999-2012,NaN,NaN,NaN,NaN,NaN
10,November,264.8,18,25.6,19.9,22.8,22.4,21.3,20.8,24.7,...,3.0,6,5,7,anayrizal1999-2012,NaN,NaN,NaN,NaN,NaN
11,December,202.3,16,24.3,19.1,21.7,21.5,20.4,19.9,23.3,...,4.0,6,1,1,anayrizal1999-2012,NaN,NaN,NaN,NaN,NaN


In [99]:
def pickleOut(obj, fname):
    fout =  open(fname, 'wb')
    pickle.dump(obj, fout)
    fout.close()

In [96]:
pickleOut(dfFinal, 'dfprecip.pkl')

In [101]:
dfFinal.head(1000)

,month,rainfall_amount,rainfall_number_of_rd,temp_max,temp_min,temp_mean,temp_dry_bulb,temp_wet_bulb,temp_dew_pt,vapor_pressure_mbs,...,wind_spd_mps,cloud_amount_okta,number_of_days_with_tstm,number_of_days_with_ltng,location,rel_hum,mslp,vapor_pressure,,wind_spd
0,January,250.2,21,28.3,21.9,25.1,25.5,23.6,22.9,27.8,...,4.0,7,0,0,alabatquez1999-2010,NaN,NaN,NaN,NaN,NaN
1,February,151.8,14,28.9,21.9,25.4,25.8,23.8,23.0,28.1,...,3.0,6,0,0,alabatquez1999-2010,NaN,NaN,NaN,NaN,NaN
2,March,127.4,11,30.2,22.6,26.4,26.7,24.5,23.7,29.2,...,3.0,5,2,1,alabatquez1999-2010,NaN,NaN,NaN,NaN,NaN
3,April,91.9,9,31.8,23.4,27.6,28.0,25.4,24.5,30.6,...,2.0,5,5,3,alabatquez1999-2010,NaN,NaN,NaN,NaN,NaN
4,May,114.1,11,32.8,23.8,28.3,28.7,26.0,25.1,31.7,...,2.0,6,14,12,alabatquez1999-2010,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,September,482.8,21,26.2,20.4,23.3,23.0,22.1,21.8,26.1,...,3.0,7,14,13,anayrizal1999-2012,NaN,NaN,NaN,NaN,NaN
9,October,276.5,18,26.2,20.4,23.3,22.9,21.8,21.4,25.4,...,3.0,6,10,13,anayrizal1999-2012,NaN,NaN,NaN,NaN,NaN
10,November,264.8,18,25.6,19.9,22.8,22.4,21.3,20.8,24.7,...,3.0,6,5,7,anayrizal1999-2012,NaN,NaN,NaN,NaN,NaN
11,December,202.3,16,24.3,19.1,21.7,21.5,20.4,19.9,23.3,...,4.0,6,1,1,anayrizal1999-2012,NaN,NaN,NaN,NaN,NaN


In [105]:
import qgrid

In [106]:
qgrid.show_grid(dfFinal, show_toolbar=True)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…